In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Flatten,Dense,Dropout,MaxPooling2D

import keras.backend as K

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

In [ ]:
import cv2

In [ ]:
TRAIN_SPLIT = 0.7

In [ ]:
MODEL_NAME = "keras_bbox_regression_model.h5"

#### The bbox regression data is created by the following repo:

`git clone https://github.com/sanjeev309/synthetic_bbox_regression_db_tool`

Run `python3 main.py` to generate **data.npy** and **target.npy**

In [ ]:
X = np.load('data.npy')
y = np.load('target.npy')

#### Inspect data

In [ ]:
X.shape

In [ ]:
plt.imshow(X[0,:,:,-1],cmap=cm.gray, vmin=0, vmax=1)

In [ ]:
y[0]

In [ ]:
np.max(y)

In [ ]:
num_samples = X.shape[0]

#### Test / Train Split

In [ ]:
X_train = X[0: int(num_samples * TRAIN_SPLIT) - 1]
y_train = y[0: int(num_samples * TRAIN_SPLIT) - 1]

X_test = X[int(num_samples * TRAIN_SPLIT): num_samples - 1]
y_test = y[int(num_samples * TRAIN_SPLIT): num_samples - 1]

In [ ]:
print(X_train.shape)
print(X_test.shape)

#### Model 

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3,activation = 'relu', input_shape=(100,100,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=3,activation = 'relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(16, kernel_size=3,activation = 'relu',))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(4096))
model.add(Dense(64))
model.add(Dense(4))

In [ ]:
model.summary()

#### Define Custom Loss function

In [ ]:
def iou_loss(y_true, y_pred):
    # iou loss for bounding box prediction
    # input must be as [x1, y1, x2, y2]
    
    # AOG = Area of Groundtruth box
    AoG = K.abs(K.transpose(y_true)[2] - K.transpose(y_true)[0] + 1) * K.abs(K.transpose(y_true)[3] - K.transpose(y_true)[1] + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(K.transpose(y_pred)[2] - K.transpose(y_pred)[0] + 1) * K.abs(K.transpose(y_pred)[3] - K.transpose(y_pred)[1] + 1)

    # overlaps are the co-ordinates of intersection box
    overlap_0 = K.maximum(K.transpose(y_true)[0], K.transpose(y_pred)[0])
    overlap_1 = K.maximum(K.transpose(y_true)[1], K.transpose(y_pred)[1])
    overlap_2 = K.minimum(K.transpose(y_true)[2], K.transpose(y_pred)[2])
    overlap_3 = K.minimum(K.transpose(y_true)[3], K.transpose(y_pred)[3])

    # intersection area
    intersection = (overlap_2 - overlap_0 + 1) * (overlap_3 - overlap_1 + 1)

    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())

    # loss for the iou value
    iou_loss = -K.log(iou)

    return iou_loss

#### Define Custom Metric

In [ ]:
def iou_metric(y_true, y_pred):
    # iou as metric for bounding box regression
    # input must be as [x1, y1, x2, y2]
    
    # AOG = Area of Groundtruth box
    AoG = K.abs(K.transpose(y_true)[2] - K.transpose(y_true)[0] + 1) * K.abs(K.transpose(y_true)[3] - K.transpose(y_true)[1] + 1)
    
    # AOP = Area of Predicted box
    AoP = K.abs(K.transpose(y_pred)[2] - K.transpose(y_pred)[0] + 1) * K.abs(K.transpose(y_pred)[3] - K.transpose(y_pred)[1] + 1)

    # overlaps are the co-ordinates of intersection box
    overlap_0 = K.maximum(K.transpose(y_true)[0], K.transpose(y_pred)[0])
    overlap_1 = K.maximum(K.transpose(y_true)[1], K.transpose(y_pred)[1])
    overlap_2 = K.minimum(K.transpose(y_true)[2], K.transpose(y_pred)[2])
    overlap_3 = K.minimum(K.transpose(y_true)[3], K.transpose(y_pred)[3])

    # intersection area
    intersection = (overlap_2 - overlap_0 + 1) * (overlap_3 - overlap_1 + 1)

    # area of union of both boxes
    union = AoG + AoP - intersection
    
    # iou calculation
    iou = intersection / union

    # bounding values of iou to (0,1)
    iou = K.clip(iou, 0.0 + K.epsilon(), 1.0 - K.epsilon())

    return iou 

In [ ]:
model.compile(loss=iou_loss, optimizer='sgd', metrics=[iou_metric])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10,batch_size=10)

In [ ]:
model.save(MODEL_NAME)

### Infer from the model and visualise results
---

In [ ]:
from keras.models import load_model

In [ ]:
infer_model = load_model(MODEL_NAME,custom_objects={'iou_loss':iou_loss,'iou_metric':iou_metric})

In [ ]:
# Might as well free up memory occupied by model
del model

In [ ]:
test_data_index = np.random.choice(num_samples,size=10,replace=False)
test_data = X[test_data_index]

In [ ]:
def convert_grayscale_to_rgb(gray):
    rgb_image = np.stack((gray,gray,gray),axis=2)
    return rgb_image[:,:,:,-1]

### Predict bounding box coordinates on the entire batch

In [ ]:
prediction = infer_model.predict(test_data)

scale = test_data.shape[1]
prediction = prediction * scale
prediction

In [ ]:
plt.figure(figsize=(25,25))

for i in range(test_data.shape[0]):
    image = convert_grayscale_to_rgb(test_data[i])
    
    x1,y1,x2,y2 = np.split(prediction[i],indices_or_sections=4)
    image = cv2.rectangle(image,(x1,y1),(x2,y2),(1,0,0),1)
    plt.subplot2grid((5,5),(i//2,i%2))
    plt.imshow(image)    

##### Feel like you can improve something ? Pull requests are always welcome :) 